<a href="https://colab.research.google.com/github/EduardoMoraesRitter/introducao-a-sistemas-de-recomendacao-com-python/blob/master/KNN_Gerando_recomenda%C3%A7%C3%B5es_baseado_em_um_usu%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

filmes = pd.read_csv('movies.csv')
notas = pd.read_csv('ratings.csv')

filmes.columns = ['filmeId', 'titulo', 'generos']
filmes = filmes.set_index('filmeId')
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']

In [84]:
#filtar um usuario, pegar duas coluna, colocar com indice
notas_usuario = notas.query('usuarioId==1')
notas_usuario = notas_usuario[['filmeId', 'nota']].set_index("filmeId")
notas_usuario.head()

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0


#Parametros para teste

In [0]:
def notas_usuario(usuarioId):
    notas_usuario = notas.query('usuarioId==%d' % usuarioId)
    notas_usuario = notas_usuario[['filmeId', 'nota']].set_index("filmeId")
    return notas_usuario

In [0]:
def distancia_usuarios(uId1,uId2, minimo=5):
  u1N = notas_usuario(uId1)
  u2N = notas_usuario(uId2)
  diferencas = u1N.join(u2N, lsuffix="u_esq", rsuffix="u_dir").dropna()
  if(len(diferencas) < minimo):
    return None
  distancia_usu1_usu2 = distancia_vetor(diferencas['notau_esq'], diferencas['notau_dir'])
  return [uId1, uId2, distancia_usu1_usu2]

In [0]:
def distancia_todos(voce, numero_usuario_analizar = None):
  todos_usuarios = notas['usuarioId'].unique()
  if numero_usuario_analizar:
     todos_usuarios = todos_usuarios[:numero_usuario_analizar]
  distancias = [distancia_usuarios(voce,usuario_id) for usuario_id in todos_usuarios]
  distancias = list(filter(None, distancias))
  distancias = pd.DataFrame(distancias, columns = ["voce", "outro", "distancia"])
  return distancias

In [88]:
def mais_proxima(voce_id, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id)
  return distancias
mais_proxima(1, numero_usuario_analizar=5)

,voce,distancia
outro,,
5,1,3.741657
3,1,8.200610
4,1,11.135529


#pegar o usuario mais similar, pegar os filmes que ele assistiu e vc nao 

In [89]:
voce = 1
notas_voce = notas_usuario(voce)
filmes_voce = notas_voce.index
similares = mais_proxima(voce, numero_usuario_analizar=5)
#pega a linha iloc, pega o name
similar = similares.iloc[0].name
notas_similar = notas_usuario(similar)
#remover os filmes q vc assistiu da lista, que vc viu q ele nao viu
notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
notas_similar = notas_similar.sort_values("nota", ascending=False)
notas_similar.head()

,nota
filmeId,
475,5.0
595,5.0
58,5.0
594,5.0
290,5.0


In [0]:
def sugere_para(voce, numero_usuario_analizar=None):
  notas_voce = notas_usuario(voce)
  filmes_voce = notas_voce.index
  similares = mais_proxima(voce, numero_usuario_analizar=numero_usuario_analizar)
  similar = similares.iloc[0].name
  notas_similar = notas_usuario(similar)
  notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
  recomendacoes = notas_similar.sort_values("nota", ascending=False)
  return recomendacoes.join(filmes)

In [91]:
sugere_para(1).head()

,nota,titulo,generos
filmeId,,,
8636,5.0,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX
58559,5.0,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
33794,5.0,Batman Begins (2005),Action|Crime|IMAX
4993,5.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
5349,5.0,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller
